In [ ]:
import torch
import numpy as np
import math

In [ ]:
device = torch.device("cuda")

In [ ]:
def get_memory_size(input, sparse=False):
    if sparse:
        return (input._values().element_size() * input._values().nelement() + 
               input._indices().element_size() * input._indices().nelement() + 8) * 1e-9
    return input.element_size() * input.nelement() * 1e-9 # in GB

def get_sparsity(input):
    return 1 - input._nnz()/input.nelement()

# Graph

In [ ]:
from gechebnet.graph.graph import SE2GEGraph

In [ ]:
xi, eps = 1., 1.
se2_graph = SE2GEGraph(
    nx=4,
    ny=4,
    ntheta=1,
    knn=8,
    sigmas=(xi / eps, xi, 1.0),
    weight_kernel=lambda sqdistc, sigmac: torch.exp(-sqdistc / sigmac),
)

In [ ]:
se2_graph.set_laplacian(norm=True, device=device)

In [ ]:
from gechebnet.model.convolution import ChebConv

In [ ]:
cheb_conv = ChebConv(se2_graph.laplacian, 1, 1, 2, ).to(device)

In [ ]:
L1 = cheb_conv.laplacian.clone()

In [ ]:
L2 = cheb_conv.laplacian.clone()

In [ ]:
L1

In [ ]:
L2

In [ ]:
se2_graph.set_sparse_laplacian(norm=True, on="edges", rate=0.5, device=device)

In [ ]:
x = torch.rand(1, 1, 16).to(device)

In [ ]:
cheb_conv(x, se2_graph.laplacian)

In [ ]:
from torch.nn import BatchNorm1d, ReLU, Module

class ResidualBlock(Module):
    def __init__(self, in_channels, out_channels, K):
        super(ResidualBlock, self).__init__()
        
        self.bn1 = BatchNorm1d(in_channels)
        self.relu1 = ReLU(inplace=True)
        self.conv1 = ChebConv(in_channels, out_channels, K, bias=True)
        self.bn2 = BatchNorm1d(out_channels)
        self.relu2 = ReLU(inplace=True)
        self.conv2 = ChebConv(out_channels, out_channels, K, bias=True)

        self.equalInOut = in_channels == out_channels
        
        if not self.equalInOut:
            self.convShortcut = ChebConv(in_channels, out_channels, K=1, bias=False)

    def forward(self, x, laplacian):
        x = self.bn1(x)
        out = self.relu1(self.conv1(x, laplacian))
        if self.equalInOut:
            return self.relu2(x + self.conv2(self.bn2(out), laplacian))
        return self.relu2(self.convShortcut(x) + self.conv2(self.bn2(out), laplacian))

In [ ]:
500 * 16

In [ ]:
res_block = ResidualBlock(1, 1, 2).to(device)
res_block(x, se2_graph.laplacian)

In [ ]:
from gechebnet.model.utils import ResidualBlock, NetworkBlock

res_block = NetworkBlock(1, 2, 2, ResidualBlock, 2).to(device)

In [ ]:
x = torch.ones(1, 1, 16, device=device)
laplacian = se2_graph.laplacian.to(device)

In [ ]:
laplacian

In [ ]:
res_block(x, laplacian)

In [ ]:
c = ChebConv(1, 1, 2)
c.weight

# Data

In [ ]:
x = torch.rand(15, 3, se2_graph.num_nodes, device=device)

In [ ]:
get_memory_size(x)

## Wide Group Equivariant ChebNet

In [ ]:
from gechebnet.model.chebnet import WideGEChebNet

In [ ]:
model = WideGEChebNet(in_channels=1, out_channels=10, K=2, graph=se2_graph, depth=8, widen_factor=2)
model = model.to(device)

In [ ]:
model.capacity

In [ ]:
model(x)

In [ ]:
model.graph.laplacian

In [ ]:
se2_graph.set_sparse_laplacian(on="edges", rate=0.4, norm=True, device=device)

In [ ]:
model.graph.laplacian

In [ ]:
from torch import nn
m = nn.AdaptiveMaxPool1d(5)
input = torch.randn(1, 64, 8)
output = m(input)
output.shape

## Wide Residual Group Equivariant ChebNet

In [ ]:
from gechebnet.model.reschebnet import WideResGEChebNet

In [ ]:
model = WideResGEChebNet(in_channels=1, out_channels=10, K=5, graph=se2_graph, depth=26, widen_factor=2)
model = model.to(device)
model

In [ ]:
model.capacity

In [ ]:
model(x)

In [ ]:
class A:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
        
class B:
    def __init__(self, b, c):
        self.b = b
        self.c = c

In [ ]:
object_B = B(42, 1)
object_A = A(12, object_B, 100)

In [ ]:
object_A.b.b

In [ ]:
object_B.b = 12

In [ ]:
object_A.b.b